In [34]:

# 코드 작성일 : 24.7.4 -> 대표메뉴
import os
if 'original_dir' not in globals() :
    original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
exec(open('setup/default.py').read())

/Users/user/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [9]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [24]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    return text

# 데이터프레임에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)



In [35]:
first_menu_list = menu_w_sold_out.groupby(['first_menu'],dropna=False).size().to_frame('size').reset_index().sort_values(by='size',ascending=False)

In [53]:
# 비교할 텍스트
target_text = '고기미역국'

# 유사도 계산하여 가장 유사한 메뉴 찾기
first_menu_list ['similarity'] = first_menu_list ['first_menu'].apply(lambda x: fuzz.token_set_ratio(x, target_text))
most_similar_menu = first_menu_list.loc[first_menu_list ['similarity'].idxmax()]

In [54]:
first_menu_list.sort_values(by='similarity',ascending=False).iloc[:10,:]

,first_menu,size,similarity
382,쇠고기미역국,1,91
361,소고기미역국,14,91
38,고기국수,1,67
659,황태미역국,6,60
520,제주고기국수,1,55
370,소고기쌀국수,2,55
207,맑은소고기무국,1,50
474,우불고기비빔국수,1,46
374,소고기청국장찌개,1,46
20,경상도소고기무국,5,46
